In [1]:
include("./helper_functions.jl")
include("./q_properties.jl")
include("./q_matroids.jl")
include("./enumeration.jl")
include("./optimizied_enumeration.jl")
include("./database.jl")
using DataFrames
using SQLite

# **$q$-Matroids in Julia/Oscar**

<h3>

Sebastian Degen, Bielefeld University, CRC TRR 358
</h3>


<h2>

**Outline:**


1. Basics about $q$-matroids
2. Representable $q$-matroids

</h2>

## **1. Basics about $q$-matroids**

<h3>

The theory of $q$-matroids was first introduced by Jurrius and Pellikaan in 2018.

</h3>

### **Motivation**

<h3>

**Slogan:** ***$\quad q$-matroids are the $q$-analogues of matroids.***


<h3>

### **$q$-Analogue**


<h4>

Theory of generalizing finite set related concepts to concepts related to $\mathbb{F}_q^n$.

***Finite sets $[n]$ ------> Finite dim. vector spaces over $\mathbb{F}_q$.***


</h4>

<h5>

**Motivating Example:**

$$

\text{Num. of } k\text{-subsets of } [n] = {n\choose k}\; --->\; {n\choose k}_q = \text{Num. of } k\text{-spaces of } \mathbb{F}_q^n. 

$$

</h5>

In [2]:
# q=2,n=3,k=2
q_binomcoeff(2,3,2)

7

### **$q$-Matroids**

<h4>

**Notation:** $\quad\mathcal{L}(\mathbb{F}_q^n)\hat{=}$ subspace lattice, i.e., set of all subspaces of $\mathbb{F}_q^n$ ordered by $\leq$.

<!-- $V\leq W\hat{=}$ the subspace $V$ is contained in the subspace $W$. -->


</h4>

<h4>

**Definition:** **(by Rank-function)** $\;\mathcal{M}=(E,\rho)$, where:

<ul>

<li>

$E=\mathbb{F}_q^n$.

</li> 

<li>

Function $\rho:\mathcal{L}(E)\rightarrow \mathbb{Z}_{\geq 0}$ satisfying: for all $V,W\in\mathcal{L}(E)$

<ul>

<li> 

**(R1)** $\quad 0\leq\rho(V)\leq\dim(V)$.

</li>

<li> 

**(R2)** $\quad\text{If }V\leq W$ $\Rightarrow$ $\rho(V)\leq\rho(W)$.

</li>

<li>

**(R3)** $\quad\rho(V+W)+\rho(V\cap W)\leq\rho(V)+\rho(W)$.

</li>

</ul>
</li>

</ul>
</h4>

<h4>  

**Ex:** $\;\rho(V)=\dim(V)$ for all $V\in\mathcal{L}(E)$.

</h4>

<h4>

**Definition:** **(by Bases)** $\mathcal{M}=(E,\mathcal{B})$, where:

* $E=\mathbb{F}_q^n$.
* Collection $\mathcal{B}\subseteq\mathcal{L}(E)$ satisfying: $\;\forall B_1,B_2\in\mathcal{B}$
    + **(B1)**$\quad\mathcal{B}\not=\emptyset$.
    + **(B2)**$\quad\text{If }B_1\leq B_2$ $\Rightarrow$ $B_1=B_2$.
    + **(B3)**$\quad\text{For all }B_1,B_2\in\mathcal{B}$ $\text{and for all codim. one } A\leq B_1$<br>$\quad\text{ there ex. a codim. one } X\leq E \text{ s.t. } A\leq X,B_2\not\leq X$ <br> $\quad\;\text{and } A+x\in\mathcal{B}\text{ for all one-spaces } x\leq E, x\not\leq X$.
    
</h4>

<h4>

**Ex:** $\;\quad\mathcal{B}=\{E\}$.

</h4>

| Rank-function | Bases |
|:-|:-|
| $\mathcal{M}=(E,\rho)$, where <ul><li> $E=\mathbb{F}_q^n$ </li><li> $\rho:\mathcal{L}(E)\rightarrow \Z_{\geq 0}$ satisfying: $\;\forall V,W\in\mathcal{L}(E)$ </li></ul>| $\mathcal{M}=(E,\mathcal{B})$, where <ul> <li> $E=\mathbb{F}_q^n$ </li><li> $\mathcal{B}\subseteq\mathcal{L}(E)$ satisfying: $\;\forall B_1,B_2\in\mathcal{B}$</li></ul>
|<ul><li>**(R1)**$\quad 0\leq\rho(V)\leq\dim(V)$.</li><br><li>**(R2)**$\quad\text{If }V\leq W$ $\Rightarrow$ $\rho(V)\leq\rho(W)$.</li><br><li>**(R3)**$\quad\rho(V+W)+\rho(V\cap W)\leq\rho(V)+\rho(W)$.<br><br><br></li></ol>|<ul><li>**(B1)**$\quad\mathcal{B}\not=\emptyset$.</li><br><li>**(B2)**$\quad\text{If }B_1\leq B_2$ $\Rightarrow$ $B_1=B_2$.</li><br><li>**(B3)**$\quad\text{For all }B_1,B_2\in\mathcal{B}$ $\text{and for all codim. one } A\leq B_1$<br>$\quad\text{ there ex. a codim. one } X\leq E \text{ s.t. } A\leq X,B_2\not\leq X$ <br> $\quad\;\text{and } A+x\in\mathcal{B}\text{ for all one-spaces } x\leq E, x\not\leq X$.</li></ol>|
|**Exp:**$\quad\rho(V)=\dim(V)$ for all $V\in\mathcal{L}(E)$.|**Exp:**$\quad\mathcal{B}=\{E\}$.|

<h4>

**Remark:**
1. The above descriptions of $\mathcal{M}$ are $q$-cryptomorphisms, i.e., equivalent axiomatic definitions of the same $q$-matroid.
    * Rank-function Def. ------> Bases Def.: $$\mathcal{B}_\rho:=\{X\in\mathcal{L}(E)\;|\;\rho(X)=\dim(X)=\rho(E)\}.$$
    * Bases Def. ------> Rank-function Def.: $$\rho_\mathcal{B}(X)=\max\{\dim(X\cap B)\;|\;B\in\mathcal{B}\}.$$
2. Every space in $\mathcal{B}$ has the same dimension.
3. The *rank of $\mathcal{M}$* is the value: $$\text{rank}(\mathcal{M})=\rho(E)\; \text{ equivalently }\; \text{rank}(\mathcal{M})=\dim(B)\text{ for any }B\in\mathcal{B}.$$

</h4>

### **$q$-Matroids: Julia/Oscar**

<h3>

```
struct Q_Matroid
    groundspace::fpMatrix           # groundspace as id-matrix.
    bases::AbstractVector{fpMatrix} # bases of the q-matroid.
end

```
</h3>

<h4>

**Note:** <br>
For every dimension $0\leq k\leq n$, we always represent a $k$-space $V\in\mathcal{L}(\mathbb{F}_q^n)$ by a generator matrix $G\in\mathbb{F}_q^{k\times n}$, i.e., $V=\text{rowspace}_{\mathbb{F}_q}(G)$. 

</h4>

In [3]:
# Check if the given collection is a bases collection
dim, subdim, field = 3, 1, GF(2)
A = subspaces_fix_dim(field,subdim,dim)
B = [matrix(field,[1 0 0])]
Are_q_matroid_bases(A),Are_q_matroid_bases(B)

(true, false)

In [4]:
# Construct the q-matroid
groundspace = matrix_space(field,dim,dim)(1)
M = Q_Matroid(groundspace,A)

Q-Matroid of rank 1 in 3-dim. vector-space over the Finite field of characteristic 2

<h4>

The $q$-matroid `M` is called *uniform $q$-matroid of rank $1$ in dim. $3$*.

</h4>

<h4>

**Definition:** <br>
For $0\leq k\leq n$ *the uniform $q$-matroid* is defined as $\;\mathcal{U}_{k,n}(\mathbb{F}_q)=(\mathbb{F}_q^n,\rho)$.

* **Rank-function:** $\quad\rho(V)=\min\{k,\dim(V)\}\;$ for all $V\in\mathcal{L}(\mathbb{F}_q^n)$.
* **Bases:** $\quad\mathcal{B}=\{\text{all }k\text{-spaces}\}$.



</h4>

In [5]:
# Construct uniform q-matroid
field, k, n = GF(2), 1, 3
UQM1 = Uniform_q_matroid(field,k,n)

Q-Matroid of rank 1 in 3-dim. vector-space over the Finite field of characteristic 2

In [6]:
# Check the ranks of a few subspaces
V_1 = matrix(field,[0 0 0])
V_2 = matrix(field,[0 1 0])
V_3 = matrix(field,[1 0 1;0 1 0])
Q_Matroid_Ranks(M,V_1),Q_Matroid_Ranks(M,V_2),Q_Matroid_Ranks(M,V_3)

(0, 1, 1)

### **Associated notions**

<h4>

Let $\mathcal{M}=(E,\rho)$ a $q$-matroid and $\mathcal{B}$ its bases collection.



|Notion|In terms of *rank* $\rho$|In terms of *bases* $\mathcal{B}$|
|:-:|:-|:-|
|**Independent space**|$V\in\mathcal{L}(E)$ s.t. $\rho(V)=\dim(V).$|Space contained in some bases.|
|**Dependent space**|$V\in\mathcal{L}(E)$ s.t. $\rho(V)<\dim(V).$|Space contained in no bases.|
|**Circuit**|$C\in\mathcal{L}(E)$ s.t. every subspace is indep.|Minimal dep. space w.r.t. $\leq$|
|**Flat**|$F\in\mathcal{L}(E)$ s.t. $\rho(F+x)>\rho(F)$, for all one-spaces $x\not\leq F$.| No "nice" description|

<!-- |**Loop**|$x\in\mathcal{L}(E)$ s.t. $\dim(x)=1$ and $x$ dep.|One-space contained in no bases.| -->



</h4>

<h4>

**Note:** All these notions provide $q$-cryptomorphisms of $\mathcal{M}$.
<!-- * *Loops come in subspace*, i.e., if $x,y$ are loops then $\rho(x+y)=0$. -->

</h4>

### **Associated notions: Julia/Oscar**

In [7]:
# UQM2
field, k, n = GF(2), 2, 3
UQM2 = Uniform_q_matroid(field,k,n)

Q-Matroid of rank 2 in 3-dim. vector-space over the Finite field of characteristic 2

In [8]:
# Indep. spaces
length(Q_Matroid_Independentspaces(UQM2))

15

In [9]:
# Dep. spaces
Q_Matroid_Dependentspaces(UQM2)

1-element Vector{fpMatrix}:
 [1 0 0; 0 1 0; 0 0 1]

In [10]:
# Loops
#Q_Matroid_Loopspace(UQM2)

In [11]:
# Circuits
Q_Matroid_CircuitsV2(UQM2)

1-element Vector{fpMatrix}:
 [1 0 0; 0 1 0; 0 0 1]

In [12]:
# Flats
Q_Matroid_Flats(UQM2)

9-element Vector{fpMatrix}:
 [0   0   0]
 [1   0   0]
 [0   1   0]
 [1   1   0]
 [0   0   1]
 [1   0   1]
 [0   1   1]
 [1   1   1]
 [1 0 0; 0 1 0; 0 0 1]

## **2. Representable $q$-matroids**

### **Motivation**

<h4>

**Algebraic coding theory**

- Constructing error-control codes for data transmissions.
- Some can be modeled via $\mathbb{F}_{q^m}$-linear subspaces of $\mathbb{F}_{q^m}^n$ (called *rank-metric codes*).
- These define representable $q$-matroids.
- Properties of these $q$-matroid help to understand the associated codes.

</h4>

### **Representable $q$-matroids**

<h4>

**Proposition:** (Lueressen, Jany 2022)<br>
Let $\mathcal{C}\leq\mathbb{F}_{q^m}^n$ be a $k$-dim. rank-metric code and $G\in\mathbb{F}_{q^m}^{k\times n}$. Then the function $\rho_G:\mathcal{L}(\mathbb{F}_q^n)\rightarrow\mathbb{Z}_{\geq 0}$ given by $$\rho_G(V)=\text{rk}_{\mathbb{F}_{q^m}}(GY^T),\text{ where }V=\text{rowspace}_{\mathbb{F}_{q}}(Y)$$ is a $q$-rank function. We call $\mathcal{M}_G=(\mathbb{F}_q^n,\rho_G)$ *$q$-matroid associated to $G$*.

</h4>

<h4>

**Definition:** <br>
A $q$-matroid $\mathcal{M}=(\mathbb{F}_q^n,\rho)$ of rank $k$ is *$\mathbb{F}_{q^m}$-representable* if there ex. $m\geq 1$ and $G\in\mathbb{F}_{q^m}^{k\times n}$ s.t. $\mathcal{M}_G=\mathcal{M}.$

</h4>

### **Representable $q$-matroids: Julia/Oscar**

In [13]:
# Define q-matroid from matrix (QM2)
# Ext_F2,x = FiniteField(2,1,"x")
# Mat2 = matrix(Ext_F2,[1 0 0;0 1 0])
# QM2 = q_matroid_from_matrix(Mat2)

In [14]:
# Bases QM2
#QM2.bases

In [15]:
# Check if QM1==QM2
#Set(QM1.bases) == Set(QM2.bases)

In [16]:
# q-matroid from matrix
Ext_F3,y = FiniteField(2,3,"y")
Mat3 = matrix(Ext_F3,[1 0 y;0 1 y^2])
QM = q_matroid_from_matrix(Mat3)

Q-Matroid of rank 2 in 3-dim. vector-space over the Finite field of characteristic 2

In [17]:
# Bases QM
QM.bases 

7-element Vector{fpMatrix}:
 [1 0 0; 0 1 0]
 [1 0 0; 0 0 1]
 [1 0 0; 0 1 1]
 [0 1 0; 0 0 1]
 [1 0 1; 0 1 0]
 [1 1 0; 0 0 1]
 [1 0 1; 0 1 1]

<h4>

The $q$-matroid `QM3` is called *uniform $q$-matroid of rank $2$ in dim. $3$*.

</h4>

<h4>

**Definition:** <br>
For $0\leq k\leq n$ the $q$-matroid $\mathcal{U}_{k,n}(\mathbb{F}_q)=(\mathbb{F}_q^n,\{\text{all }k\text{-spaces}\})$ is called *the uniform $q$-matroid*.

</h4>

<h4>

**Proposition:** $\quad\mathcal{U}_{k,n}(\mathbb{F}_q)$ is $\mathbb{F}_{q^m}$-representable $\Longleftrightarrow$ $m\geq n$.

</h4>

In [18]:
# Construct uniform q-matroid
# field = GF(2)
# k = 1
# n = 3
# UQM = Uniform_q_matroid(field,k,n)

In [19]:
# Circuits
#Q_Matroid_CircuitsV2(UQM)

In [20]:
# Flats
# Q_Matroid_Flats(UQM)

### **Representability Algorithm**

<h3>
<center>

**Question:** Given a $q$-matroid can we decide whether it is representable?

</center> 

</h3>

<h3>
<center>

**Answer:** Yes, there is an algorithm to do so. (Kühne and D.)

</center>

</h3>

<h4>

**Algo:** $\quad\mathcal{M}=(\mathbb{F}_q^n,\rho)$ of rank $k$.

<!-- * *Setup:*   $\mathcal{M}=(\mathbb{F}_q^n,\rho)$ of rank $k$. -->
* *Input:*   $\mathcal{B}$ of $\mathcal{M}$.
* *Output:*  `True` or `False`
* *Idea:*
    - Check if ex. $\tilde{G}\in\mathbb{F}_{q^m}^{k\times n}$ for some $m\geq 1$ s.t. $\rho(V)=\text{rk}(\tilde{G}V^T)$.
    - Must have:
        + $\det(\tilde{G}B^T)\not= 0$ for all $B\in\mathcal{B}$.
        + $\det(\tilde{G}D^T)= 0$ for all $D\in\mathcal{NB}=$ $k$-dim. non-bases.
    - Consider $S=\bar{\mathbb{F}_q}[x,z]=\bar{\mathbb{F}_q}[x_{i,j},z\;|\;1\leq i\leq k,1\leq j\leq n]$ and $G=(x_{i,j})\in(\bar{\mathbb{F}_q}[x,z])^{k\times n}$. 
        + For all $B\in\mathcal{B},D\in\mathcal{NB}$ define: $$P_B(x,z):=\det(GB^T)\;\text{ and }\; P_D(x,z):=\det(GD^T).$$
        + Define: $$R(x,z)=z \cdot \Big(\prod_{B\in\mathcal{B}}P_B\Big)-1\;\text{ and }\; I:=(R,P_D\;|\;D\in\mathcal{NB}).$$
    - Check if $\mathcal{V}(I)$ is empty: $$\mathcal{V}(I)=\emptyset\Longleftrightarrow 1\in I\quad\text{by weak Hilbert Nullstellen-Satz}.$$

</h4>

### **Algorithm: Julia/Oscar**

In [21]:
# UQM2
Ext_F3,y = FiniteField(2,3,"y")
Mat3 = matrix(Ext_F3,[1 0 y;0 1 y^2])
QM = q_matroid_from_matrix(Mat3)    # = UQM2

Q-Matroid of rank 2 in 3-dim. vector-space over the Finite field of characteristic 2

In [22]:
# Check for representability
Is_representable(QM)

("Q-Matroid is representable!!", ideal(x1^4*x2^2*x7 + x1^4*x2*x7 + x1^2*x2^4*x7 + x1^2*x2*x7 + x1*x2^4*x7 + x1*x2^2*x7 + 1))

In [23]:
# QM2
field, dim, qrank= GF(2), 4, 2 
Groundspace = matrix_space(field,dim,dim)(1) 
All_two_spaces = subspaces_fix_dim(field,qrank,dim)
Leftoutspaces = [matrix(field,[1 0 0 0;0 1 0 0]),
                 matrix(field,[0 0 1 0;0 0 0 1]),
                 matrix(field,[1 0 0 1;0 1 1 0]),
                 matrix(field,[1 0 1 0;0 1 1 1])]
Bases = [X for X in All_two_spaces if !(X in Leftoutspaces)]
QM2 = Q_Matroid(Groundspace,Bases)

Q-Matroid of rank 2 in 4-dim. vector-space over the Finite field of characteristic 2

In [24]:
# Check for representability
Is_representable(QM2)

("Q-Matroid is not representable!!", ideal(1))